In [48]:
pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install pingouin

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [50]:
pip install --upgrade pingouin

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


## 라이브러리

In [30]:
import pandas as pd
import numpy as np
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor

## 데이터 불러오기

In [21]:
path = './data/'

df = pd.read_csv(path+'전세가격예측프로젝트_월별데이터_총병합.csv')

In [22]:
df

,Year,Month,Region_Name,Region_Code,Building_Use,Sell_Price,Sell_BA,Sell_PPA,Sell_PPP,Sell_Count,...,JS_Count,CR,BA_CR,PPA_CR,IR,UR,LC_index,CA_index,TC_index,SDT_index
0,2012,7,강남구,11680,단독다가구,316562.50,344.92,943.75,285.48,8,...,135,3.89,19.62,22.31,3.0,3.1,74.5,81.5,78.5,102.461258
1,2012,7,강남구,11680,아파트,82549.28,81.56,1023.00,309.46,133,...,750,46.13,101.15,44.26,3.0,3.1,74.5,81.5,78.5,102.461258
2,2012,7,강남구,11680,연립다세대,28362.79,48.58,565.35,171.02,38,...,140,61.63,109.47,61.95,3.0,3.1,74.5,81.5,78.5,102.461258
3,2012,7,강남구,11680,오피스텔,25309.88,39.07,671.99,203.28,42,...,10,157.65,227.39,62.87,3.0,3.1,74.5,81.5,78.5,102.461258
4,2012,7,강동구,11740,단독다가구,103757.14,336.59,480.88,145.47,21,...,217,7.61,16.44,33.75,3.0,3.1,74.5,81.5,78.5,102.461258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12185,2022,8,중구,11140,오피스텔,33270.39,29.85,1108.02,335.18,162,...,72,74.87,104.46,74.52,2.5,2.1,109.0,109.0,110.3,87.677816
12186,2022,8,중랑구,11260,단독다가구,113917.78,221.65,517.06,156.41,81,...,271,11.79,19.67,66.34,2.5,2.1,109.0,109.0,110.3,87.677816
12187,2022,8,중랑구,11260,아파트,54521.62,58.36,989.66,299.37,74,...,414,53.66,112.94,44.39,2.5,2.1,109.0,109.0,110.3,87.677816
12188,2022,8,중랑구,11260,연립다세대,33390.06,48.04,743.72,224.98,327,...,276,72.34,78.37,95.16,2.5,2.1,109.0,109.0,110.3,87.677816


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12190 entries, 0 to 12189
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          12190 non-null  int64  
 1   Month         12190 non-null  int64  
 2   Region_Name   12190 non-null  object 
 3   Region_Code   12190 non-null  int64  
 4   Building_Use  12190 non-null  object 
 5   Sell_Price    12190 non-null  float64
 6   Sell_BA       12190 non-null  float64
 7   Sell_PPA      12190 non-null  float64
 8   Sell_PPP      12190 non-null  float64
 9   Sell_Count    12190 non-null  int64  
 10  JS_Price      12190 non-null  float64
 11  JS_BA         12190 non-null  float64
 12  JS_PPA        12190 non-null  float64
 13  JS_PPP        12190 non-null  float64
 14  JS_Count      12190 non-null  int64  
 15  CR            12190 non-null  float64
 16  BA_CR         12190 non-null  float64
 17  PPA_CR        12190 non-null  float64
 18  IR            12190 non-nu

## 다중공선성 확인

In [50]:
independent_variables = ['Year', 'Month', 'Sell_Price', 'Sell_BA', 'Sell_PPA', 'Sell_PPP', 'Sell_Count',
        'JS_BA', 'JS_PPA', 'JS_PPP', 'JS_Count', 'CR', 'BA_CR',
       'PPA_CR', 'IR', 'UR', 'LC_index', 'CA_index', 'TC_index', 'SDT_index']

In [51]:
X = df[independent_variables]

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [52]:
vif_data

,Variable,VIF
0,Year,4.549082e+03
1,Month,9.896703e+00
2,Sell_Price,1.710899e+01
3,Sell_BA,3.140324e+01
4,Sell_PPA,5.334148e+09
5,Sell_PPP,5.334151e+09
6,Sell_Count,3.617992e+00
7,JS_BA,2.478396e+01
8,JS_PPA,2.181755e+09
9,JS_PPP,2.181751e+09


## Regression_analysis

In [53]:
df.columns

Index(['Year', 'Month', 'Region_Name', 'Region_Code', 'Building_Use',
       'Sell_Price', 'Sell_BA', 'Sell_PPA', 'Sell_PPP', 'Sell_Count',
       'JS_Price', 'JS_BA', 'JS_PPA', 'JS_PPP', 'JS_Count', 'CR', 'BA_CR',
       'PPA_CR', 'IR', 'UR', 'LC_index', 'CA_index', 'TC_index', 'SDT_index'],
      dtype='object')

In [57]:
regression_js = pg.linear_regression(df[['Year', 'Month', 'Sell_Price', 'Sell_BA', 'Sell_PPA', 'Sell_PPP', 'Sell_Count',
       'JS_BA', 'JS_PPA', 'JS_PPP', 'JS_Count', 'CR', 'BA_CR',
       'PPA_CR', 'IR', 'UR', 'LC_index', 'CA_index', 'TC_index', 'SDT_index']], df['JS_Price'])
regression_js

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,1.574599e+06,1.115865,1.411101e+06,0.000000e+00,0.953906,0.95383,1.574596e+06,1.574601e+06
1,Year,-8.099991e+02,0.873097,-9.277312e+02,0.000000e+00,0.953906,0.95383,-8.117105e+02,-8.082877e+02
2,Month,-5.766827e+01,11.159475,-5.167651e+00,2.408081e-07,0.953906,0.95383,-7.954262e+01,-3.579393e+01
3,Sell_Price,-6.395922e-04,0.001316,-4.860316e-01,6.269535e-01,0.953906,0.95383,-3.219059e-03,1.939875e-03
4,Sell_BA,-2.082156e+00,1.230631,-1.691942e+00,9.068260e-02,0.953906,0.95383,-4.494388e+00,3.300758e-01
5,Sell_PPA,-4.197498e+03,2580.678733,-1.626509e+00,1.038673e-01,0.953906,0.95383,-9.256038e+03,8.610428e+02
6,Sell_PPP,1.390430e+04,8531.172211,1.629823e+00,1.031647e-01,0.953906,0.95383,-2.818151e+03,3.062676e+04
7,Sell_Count,-4.424608e+00,0.224008,-1.975203e+01,1.672510e-85,0.953906,0.95383,-4.863699e+00,-3.985517e+00
8,JS_BA,5.019820e+02,2.410517,2.082466e+02,0.000000e+00,0.953906,0.95383,4.972570e+02,5.067070e+02
9,JS_PPA,-2.412752e+03,2561.872130,-9.417926e-01,3.463175e-01,0.953906,0.95383,-7.434429e+03,2.608924e+03
